You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
%%script true
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
from aisModels_crewai import GroqLLM, ChatOllama

Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?
Hello! How are you today? Is there something I can help you with or would you like to chat?


In [5]:
llm = GroqLLM()

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [6]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Agent: Writer

In [7]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [8]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [2024-09-11 17:26:07][DEBUG]: == Working Agent: Content Planner
 [2024-09-11 17:26:07][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I'm fully focused on this task! I've gathered all the necessary information to create a comprehensive content plan document on Artificial Intelligence. Here's my final answer:

**Final Answer:**

**Content Plan Document:**

**Title:** "Unlocking the Power of Artificial Intelligence: Trends, Applications, and Future Possibilities"

**Target Audience:**

* **Demographics:** Business professionals, entrepreneurs, and individuals interested in technology and innovation
* **Interests:** Staying updated on the latest A

- Display the results of your execution as markdown in the notebook.

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

**The Future of Flight: How AI is Revolutionizing Aviation**
============================================================

**Introduction**
---------------

The aviation industry has always been at the forefront of innovation, and the latest trend to take center stage is Artificial Intelligence (AI). From predictive maintenance to air traffic control, AI is transforming the way airlines operate, making flights safer, more efficient, and more enjoyable for passengers. In this article, we'll delve into the latest trends, benefits, and challenges of AI in aviation, as well as its potential to shape the future of flight.

The integration of AI in aviation is not just a novelty, but a necessity. As the industry continues to evolve, AI is becoming an essential tool for airlines and aircraft manufacturers to stay ahead of the curve. By exploring the current state, benefits, and challenges of AI in aviation, we can better understand its potential to revolutionize the industry.

**Current State of AI in Aviation**
---------------------------------

AI is no longer a novelty in aviation; it's a reality. Airlines and aircraft manufacturers are increasingly adopting AI-powered solutions to improve their operations. Predictive maintenance, for instance, is becoming a standard practice, allowing airlines to detect potential issues before they occur, reducing downtime and increasing overall efficiency. Chatbots are also being used to enhance the passenger experience, providing personalized services and support. Moreover, AI-assisted air traffic control systems are being developed to improve safety and reduce congestion in the skies.

**Benefits of AI in Aviation**
-----------------------------

The benefits of AI in aviation are numerous and far-reaching. Firstly, AI-powered predictive maintenance can significantly improve safety by reducing the risk of mechanical failures. Secondly, AI-driven automation can increase efficiency, reducing the time and resources required for maintenance and repair. Finally, AI-powered chatbots can enhance the passenger experience, providing personalized services and support. According to a report by the FAA, AI has the potential to reduce maintenance costs by up to 20% and increase aircraft availability by up to 15%.

**Challenges and Limitations of AI in Aviation**
---------------------------------------------

While AI holds immense promise for the aviation industry, there are challenges and limitations to its adoption. Ensuring compliance with regulations and standards is a significant concern, as is managing and analyzing the vast amounts of data generated by AI systems. Integrating AI systems with existing infrastructure and processes is also a challenge that airlines and aircraft manufacturers must overcome. Moreover, addressing concerns around job displacement and cybersecurity threats is crucial for the successful implementation of AI solutions.

**Case Studies and Success Stories**
---------------------------------

Several airlines and aircraft manufacturers have successfully implemented AI solutions, demonstrating the potential of AI to transform the industry. For example, Airbus has launched an AI-powered platform for predicting and preventing aircraft component failures, while GE Aviation has developed an AI-powered engine health management system. These success stories demonstrate the benefits of AI in improving safety, efficiency, and revenue.

**Future of AI in Aviation**
---------------------------

As AI continues to evolve, we can expect to see even more innovative applications in aviation. Autonomous aircraft, for instance, could revolutionize the way we travel, making flights faster, safer, and more efficient. Urban air mobility is another area where AI is likely to play a significant role, enabling the development of electric vertical takeoff and landing (eVTOL) aircraft. According to a report by Boeing, AI has the potential to increase the efficiency of air traffic control systems by up to 30%.

**Conclusion**
----------

In conclusion, AI is transforming the aviation industry, making flights safer, more efficient, and more enjoyable for passengers. While there are challenges and limitations to its adoption, the benefits of AI in aviation are undeniable. As the industry continues to evolve, it's essential for airlines, aircraft manufacturers, and industry stakeholders to explore AI solutions for their operations. By embracing AI, we can shape the future of flight and create a safer, more efficient, and more enjoyable experience for passengers.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "AI in Aviation."
result = crew.kickoff(inputs={"topic": topic})

 [2024-09-11 17:45:49][DEBUG]: == Working Agent: Content Planner
 [2024-09-11 17:45:49][INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI in Aviation..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer!

Final Answer:

**Content Plan: AI in Aviation**

**Task 1: Latest Trends, Key Players, and Noteworthy News**

* Trends:
	+ Increased adoption of AI-powered predictive maintenance for aircraft and engines
	+ Growing use of AI-driven chatbots for customer service and passenger experience
	+ Development of AI-assisted air traffic control systems for improved safety and efficiency
* Key Players:
	+ Airbus: investing in AI research and development for aircraft design and manufacturing
	+ Boeing

In [19]:
Markdown(result.raw)

**The Future of Flight: How AI is Revolutionizing Aviation**
============================================================

**Introduction**
---------------

The aviation industry has always been at the forefront of innovation, and the latest trend to take center stage is Artificial Intelligence (AI). From predictive maintenance to air traffic control, AI is transforming the way airlines operate, making flights safer, more efficient, and more enjoyable for passengers. In this article, we'll delve into the latest trends, benefits, and challenges of AI in aviation, as well as its potential to shape the future of flight.

The integration of AI in aviation is not just a novelty, but a necessity. As the industry continues to evolve, AI is becoming an essential tool for airlines and aircraft manufacturers to stay ahead of the curve. By exploring the current state, benefits, and challenges of AI in aviation, we can better understand its potential to revolutionize the industry.

**Current State of AI in Aviation**
---------------------------------

AI is no longer a novelty in aviation; it's a reality. Airlines and aircraft manufacturers are increasingly adopting AI-powered solutions to improve their operations. Predictive maintenance, for instance, is becoming a standard practice, allowing airlines to detect potential issues before they occur, reducing downtime and increasing overall efficiency. Chatbots are also being used to enhance the passenger experience, providing personalized services and support. Moreover, AI-assisted air traffic control systems are being developed to improve safety and reduce congestion in the skies.

**Benefits of AI in Aviation**
-----------------------------

The benefits of AI in aviation are numerous and far-reaching. Firstly, AI-powered predictive maintenance can significantly improve safety by reducing the risk of mechanical failures. Secondly, AI-driven automation can increase efficiency, reducing the time and resources required for maintenance and repair. Finally, AI-powered chatbots can enhance the passenger experience, providing personalized services and support. According to a report by the FAA, AI has the potential to reduce maintenance costs by up to 20% and increase aircraft availability by up to 15%.

**Challenges and Limitations of AI in Aviation**
---------------------------------------------

While AI holds immense promise for the aviation industry, there are challenges and limitations to its adoption. Ensuring compliance with regulations and standards is a significant concern, as is managing and analyzing the vast amounts of data generated by AI systems. Integrating AI systems with existing infrastructure and processes is also a challenge that airlines and aircraft manufacturers must overcome. Moreover, addressing concerns around job displacement and cybersecurity threats is crucial for the successful implementation of AI solutions.

**Case Studies and Success Stories**
---------------------------------

Several airlines and aircraft manufacturers have successfully implemented AI solutions, demonstrating the potential of AI to transform the industry. For example, Airbus has launched an AI-powered platform for predicting and preventing aircraft component failures, while GE Aviation has developed an AI-powered engine health management system. These success stories demonstrate the benefits of AI in improving safety, efficiency, and revenue.

**Future of AI in Aviation**
---------------------------

As AI continues to evolve, we can expect to see even more innovative applications in aviation. Autonomous aircraft, for instance, could revolutionize the way we travel, making flights faster, safer, and more efficient. Urban air mobility is another area where AI is likely to play a significant role, enabling the development of electric vertical takeoff and landing (eVTOL) aircraft. According to a report by Boeing, AI has the potential to increase the efficiency of air traffic control systems by up to 30%.

**Conclusion**
----------

In conclusion, AI is transforming the aviation industry, making flights safer, more efficient, and more enjoyable for passengers. While there are challenges and limitations to its adoption, the benefits of AI in aviation are undeniable. As the industry continues to evolve, it's essential for airlines, aircraft manufacturers, and industry stakeholders to explore AI solutions for their operations. By embracing AI, we can shape the future of flight and create a safer, more efficient, and more enjoyable experience for passengers.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).